In [39]:
liked_books = ["8132407", "31147619", "29983711", "5996629", "8173788"]

In [41]:
# Compare the liked books ids with other users books id, and recommend other books based on what they liked:
csv_book_mapping={}
with open("book_id_map.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [69]:
csv_book_mapping["0"]

'34684622'

In [ ]:
# create a set where every element is unique:
overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue

        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id)

In [ ]:
rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [ ]:
# Transform recommendation in rec_lines into a dataframe:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [75]:
top_recs = recs["book_id"].value_counts().head(10)

In [76]:
top_recs = top_recs.index.values

In [77]:
# From a book id to a title (previously stored as a json file):
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [78]:
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook


In [79]:
# Books in top 10 that an user likes with the same taste as I. Probably those books I will enjoy reading:
books_titles[books_titles["book_id"].isin(top_recs)] 

,book_id,title,ratings,url,cover_image,mod_title
53027,77203,The Kite Runner,1848782,https://www.goodreads.com/book/show/77203.The_...,https://images.gr-assets.com/books/1484565687m...,the kite runner
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
463463,4671,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
615314,5470,1984,2023937,https://www.goodreads.com/book/show/5470.1984,https://images.gr-assets.com/books/1348990566m...,1984
757376,38447,The Handmaid's Tale,648783,https://www.goodreads.com/book/show/38447.The_...,https://images.gr-assets.com/books/1498057733m...,the handmaids tale
790927,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
878151,18143977,All the Light We Cannot See,498685,https://www.goodreads.com/book/show/18143977-a...,https://images.gr-assets.com/books/1451445646m...,all the light we cannot see
878545,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
1062354,29983711,Pachinko,8161,https://www.goodreads.com/book/show/29983711-p...,https://images.gr-assets.com/books/1462393298m...,pachinko
1091642,22557272,The Girl on the Train,1076144,https://www.goodreads.com/book/show/22557272-t...,https://images.gr-assets.com/books/1490903702m...,the girl on the train


In [80]:
all_recs = recs["book_id"].value_counts()

In [81]:
# Transforming all_recs to dataframe:
all_recs = all_recs.to_frame().reset_index()
# Rename columns:
all_recs.columns = ["book_id", "book_count"]

In [82]:
all_recs.head(5)

,book_id,book_count
0,29983711,1077
1,2767052,882
2,2657,874
3,18143977,851
4,3,849


In [83]:
# Merge this set with the liked books:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")

In [84]:
all_recs

,book_id,book_count,title,ratings,url,cover_image,mod_title
0,29983711,1077,Pachinko,8161,https://www.goodreads.com/book/show/29983711-p...,https://images.gr-assets.com/books/1462393298m...,pachinko
1,2767052,882,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
2,2657,874,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
3,18143977,851,All the Light We Cannot See,498685,https://www.goodreads.com/book/show/18143977-a...,https://images.gr-assets.com/books/1451445646m...,all the light we cannot see
4,3,849,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
...,...,...,...,...,...,...,...
264061,20613471,1,Happy City: Transforming Our Lives Through Urb...,55,https://www.goodreads.com/book/show/20613471-h...,https://images.gr-assets.com/books/1405301264m...,happy city transforming our lives through urba...
264062,23252419,1,"Love Me Later (Second Chances, #1)",132,https://www.goodreads.com/book/show/23252419-l...,https://images.gr-assets.com/books/1420739847m...,love me later second chances 1
264063,20263062,1,Aimless Love: New and Selected Poems,106,https://www.goodreads.com/book/show/20263062-a...,https://images.gr-assets.com/books/1404481307m...,aimless love new and selected poems
264064,23130869,1,Hologram: A Haunting,63,https://www.goodreads.com/book/show/23130869-h...,https://images.gr-assets.com/books/1409876074m...,hologram a haunting


In [85]:
# we are looking for books that user like us, enjoy, but the book may not be popular in general:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])

In [88]:
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
0,29983711,1077,Pachinko,8161,https://www.goodreads.com/book/show/29983711-p...,https://images.gr-assets.com/books/1462393298m...,pachinko,142.130744
177,32920226,315,"Sing, Unburied, Sing",4592,https://www.goodreads.com/book/show/32920226-s...,https://images.gr-assets.com/books/1499340866m...,sing unburied sing,21.608232
969,31147619,118,Homegoing,697,https://www.goodreads.com/book/show/31147619-h...,https://images.gr-assets.com/books/1491119004m...,homegoing,19.977044
156,30753987,331,The Leavers,5602,https://www.goodreads.com/book/show/30753987-t...,https://images.gr-assets.com/books/1489158974m...,the leavers,19.557479
4710,34927828,34,The Great Alone,70,https://www.goodreads.com/book/show/34927828-t...,https://images.gr-assets.com/books/1501852384m...,the great alone,16.514286
4816,35099035,33,Red Clocks,67,https://www.goodreads.com/book/show/35099035-r...,https://images.gr-assets.com/books/1494345016m...,red clocks,16.253731
1922,30288282,70,The Immortalists,319,https://www.goodreads.com/book/show/30288282-t...,https://images.gr-assets.com/books/1493015963m...,the immortalists,15.360502
7150,26856502,23,"Vengeful (Villains, #2)",35,https://www.goodreads.com/book/show/26856502-v...,https://s.gr-assets.com/assets/nophoto/book/11...,vengeful villains 2,15.114286
2008,33641244,68,The Power,315,https://www.goodreads.com/book/show/33641244-t...,https://images.gr-assets.com/books/1503003629m...,the power,14.679365
534,33280160,178,What We Lose,2250,https://www.goodreads.com/book/show/33280160-w...,https://images.gr-assets.com/books/1492235781m...,what we lose,14.081778


In [89]:
# Remove the books that have few ratings:
popular_recs = all_recs[all_recs["book_count"] > 75].sort_values("score", ascending=False)

In [90]:
#style "url" column:
def make_clickable(val):
    return '<a target=blank" href={}">Goodreads</a>'.format(val)

# style "cover_image" column:
def show_image(val):
    return '<img src="{}" width=50 </img>'.format(val)

# Prevent recommendation of a book that I already read:
popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({"url": make_clickable, 'cover_image': show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
177,32920226,315,"Sing, Unburied, Sing",4592,Goodreads,,sing unburied sing,21.608232
156,30753987,331,The Leavers,5602,Goodreads,,the leavers,19.557479
534,33280160,178,What We Lose,2250,Goodreads,,what we lose,14.081778
337,33253215,226,The Heart's Invisible Furies,3629,Goodreads,,the hearts invisible furies,14.074401
516,33621427,181,Home Fire,2390,Goodreads,,home fire,13.707531
773,30971664,138,Salt Houses,1474,Goodreads,,salt houses,12.919946
49,30688435,517,Exit West,21378,Goodreads,,exit west,12.502994
188,32283423,306,American War,7776,Goodreads,,american war,12.041667
910,33375622,123,Her Body and Other Parties,1292,Goodreads,,her body and other parties,11.709752
172,26025588,320,Behold the Dreamers,8793,Goodreads,,behold the dreamers,11.645627
